# Capstone Project - The Battle of Neighborhoods (Week 2)

## 1.Introduction
Phoenix is the capital and most populous city in Arizona. As of 2019, the city has a population of 1680992. It is also the fifth-most populous city in the United States, the largest state capital by population, and the only state capital with a population of more than one million residents.
Phoenix is the anchor of the Phoenix metropolitan area, also known as the Valley of the Sun, which in turn is part of the Salt River Valley. The metropolitan area is the 11th largest by population in the United States, with approximately 4.73 million people as of 2017. Phoenix is the seat of Maricopa County and the largest city in the state at 517.9 square miles. 
Phoenix ranks as one of the fastest-growing cities in the United States. Because of that, it is also a city with cultural diversity. Therefore, for this project, I want to find out where is the best location in Phoenix to open a Chinese restaurant. And where is the best place to stay if one prefers to eat Chinese cuisine?

## 2.Data
To answer the questions in part(1), we need data of Chinese restaurants in Phoenix. The name, location, and custom reviews of the restaurants. And use those information to determine the “best” location for open a new Chinese restaurant. In order to asset data, we use FourSquare and apply folium for visualizing a particular neighbor in which that we will observe customer "traffic" and predict an appropriate location of our restaurant in Phoenix.

#### Import tools we need.

In [100]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#from bs4 import BeautifulSoup
import os

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from folium import plugins

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

import seaborn as sns

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


#### Enter Foursquare Client Information

In [28]:
CLIENT_ID = '4V11HW4VE3OW5403FB0QNCP15DQJPGWIC5WEJYTSY33SO0JI' # your Foursquare ID
CLIENT_SECRET = 'VUMKL05U5KTKHWGZDQUE3BUTYTRSN00S4JPRR03CGYLBGYWU' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

#### Use requests to get the latitude and longitude values of Phoenix and venus of our choice.

In [91]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20210101',
    "query": "Chinese restaurant",
    "near": "Phoenix, AZ",
    "radius": 2000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

#### Transform data into json then request geocode.

In [92]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [93]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Phoenix', 'Phoenix')

In [94]:
d["suggestedBounds"], d["totalResults"]
d["geocode"]

{'what': '',
 'where': 'phoenix az',
 'center': {'lat': 33.44838, 'lng': -112.07404},
 'displayString': 'Phoenix, AZ, United States',
 'cc': 'US',
 'geometry': {'bounds': {'ne': {'lat': 33.918391, 'lng': -111.92541},
   'sw': {'lat': 33.290259999999996, 'lng': -112.324065}}},
 'slug': 'phoenix-arizona',
 'longId': '72057594043236591'}

#### Creating items of objects Chinese restaurants and their attributes, then testing it.

In [95]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 20


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '51a4187a498e776dbdf0a798',
  'name': 'Panda Express',
  'location': {'address': '401 E Jefferson St',
   'lat': 33.44568053769315,
   'lng': -112.0669919148652,
   'labeledLatLngs': [{'label': 'display',
     'lat': 33.44568053769315,
     'lng': -112.0669919148652}],
   'postalCode': '85004',
   'cc': 'US',
   'city': 'Phoenix',
   'state': 'AZ',
   'country': 'United States',
   'formattedAddress': ['401 E Jefferson St',
    'Phoenix, AZ 85004',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d145941735',
    'name': 'Chinese Restaurant',
    'pluralName': 'Chinese Restaurants',
    'shortName': 'Chinese',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-51a4187a498e776dbdf0a79

In [96]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bdb3ffb3904a593d077499e',
  'name': "Wong's Chinese Dining",
  'location': {'address': '1139 E Buckeye Rd',
   'crossStreet': 'at S 12th St',
   'lat': 33.436630125934,
   'lng': -112.05673011297367,
   'labeledLatLngs': [{'label': 'display',
     'lat': 33.436630125934,
     'lng': -112.05673011297367}],
   'postalCode': '85034',
   'cc': 'US',
   'city': 'Phoenix',
   'state': 'AZ',
   'country': 'United States',
   'formattedAddress': ['1139 E Buckeye Rd (at S 12th St)',
    'Phoenix, AZ 85034',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d145941735',
    'name': 'Chinese Restaurant',
    'pluralName': 'Chinese Restaurants',
    'shortName': 'Chinese',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '1763946',

In [97]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '401 E Jefferson St', 'lat': 33.44568053769315, 'lng': -112.0669919148652, 'labeledLatLngs': [{'label': 'display', 'lat': 33.44568053769315, 'lng': -112.0669919148652}], 'postalCode': '85004', 'cc': 'US', 'city': 'Phoenix', 'state': 'AZ', 'country': 'United States', 'formattedAddress': ['401 E Jefferson St', 'Phoenix, AZ 85004', 'United States']}
{'address': '1139 E Buckeye Rd', 'crossStreet': 'at S 12th St', 'lat': 33.436630125934, 'lng': -112.05673011297367, 'labeledLatLngs': [{'label': 'display', 'lat': 33.436630125934, 'lng': -112.05673011297367}], 'postalCode': '85034', 'cc': 'US', 'city': 'Phoenix', 'state': 'AZ', 'country': 'United States', 'formattedAddress': ['1139 E Buckeye Rd (at S 12th St)', 'Phoenix, AZ 85034', 'United States']}
{'address': '302 E Flower St', 'crossStreet': 'at N 3rd St', 'lat': 33.48586836799868, 'lng': -112.06903472542763, 'labeledLatLngs': [{'label': 'display', 'lat': 33.48586836799868, 'lng': -112.06903472542763}, {'label': 'entrance', 'lat

,uid,name,shortname,address,postalcode,lat,lng
0,51a4187a498e776dbdf0a798,Panda Express,Chinese,,85004,33.445681,-112.066992
1,4bdb3ffb3904a593d077499e,Wong's Chinese Dining,Chinese,,85034,33.436630,-112.056730
2,413ba880f964a520311b1fe3,China Chili,Chinese,,85012,33.485868,-112.069035
3,4b58bbdaf964a520206828e3,Mu Shu,Chinese,,85015,33.480710,-112.091737
4,5101ca8de4b02714c07746df,New Garden Chinese Central & Hadley,Chinese,,85041,33.439237,-112.073530


In [98]:
Phoenix_center = d["geocode"]["center"]
Phoenix_center

{'lat': 33.44838, 'lng': -112.07404}

### Print a heat map of Chinese restaurants in the center of Phoenix. 
Get coordinates of Phoenix and create folium map to help us visualize the customer flow distribution.

In [142]:
map_Phoenix = folium.Map(location=[33.44838, -112.07404], zoom_start=12)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Phoenix)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Phoenix.add_child(plugins.HeatMap(hm_data))

map_Phoenix

#### Phoenix_map_1
From the map above, we can see there are three main Chinese restaurants clusters in Phoenix. The one at the top of the map has the most customer distribution and the highest density of restaurants. Therefore, I think it is the best area for us to open a new restaurant. 

In [139]:
df

,uid,name,shortname,address,postalcode,lat,lng
0,51a4187a498e776dbdf0a798,Panda Express,Chinese,,85004,33.445681,-112.066992
1,4bdb3ffb3904a593d077499e,Wong's Chinese Dining,Chinese,,85034,33.436630,-112.056730
2,413ba880f964a520311b1fe3,China Chili,Chinese,,85012,33.485868,-112.069035
3,4b58bbdaf964a520206828e3,Mu Shu,Chinese,,85015,33.480710,-112.091737
4,5101ca8de4b02714c07746df,New Garden Chinese Central & Hadley,Chinese,,85041,33.439237,-112.073530
5,4dc1ab68b0fb1c18ee68f129,Tams Chinese And Mexican Food,Chinese,,85040,33.407044,-112.044216
6,4e89be544901b86fc2a4ecd8,Panda Express,Chinese,,85014,33.495041,-112.061321
7,59094803898bdc1a41d0bfe9,Siu Wok,Chinese,,85004,33.479070,-112.073306
8,4b634324f964a5204f6e2ae3,Long Wong's,Chinese,,85013,33.480209,-112.082618
9,5cda25a465211f002c28dccb,Panda Express,Chinese,,85041,33.391743,-112.084702


### Finding the Location of Our Restaurant
With coordinates that close to each other, we can treat the Earth as being locally flat and simply find the centroid as though they were planar coordinates. Then could simply take the average of the latitudes and the average of the longitudes to find the latitude and longitude of the centroid. Since there are only eight restaurants in the first cluster, I manually generated two lists contain the latitude and longtitude of those restaurants. And calculated the latitude and longtitude of their midpoint. 

In [140]:
lat = []
long = []
lat.append(float(df[df["name"] == "South China BBQ HK Style"]["lat"]))
lat.append(float(df[df["name"] == "Lucky Kitchen"]["lat"]))
lat.append(float(df[df["name"] == "Mu Shu"]["lat"]))
lat.append(float(df[df["name"] == "Siu Wok"]["lat"]))
lat.append(float(df[df["name"] == "China Chili"]["lat"]))
lat.append(float(df[df["name"] == "Tokyo Express"]["lat"]))
lat.append(float(df[df["name"] == "Long Wong's"]["lat"]))
lat.append(float(df[df["uid"] == "4e89be544901b86fc2a4ecd8"]["lat"]))
long.append(float(df[df["name"] == "South China BBQ HK Style"]["lng"]))
long.append(float(df[df["name"] == "Lucky Kitchen"]["lng"]))
long.append(float(df[df["name"] == "Mu Shu"]["lng"]))
long.append(float(df[df["name"] == "Siu Wok"]["lng"]))
long.append(float(df[df["name"] == "China Chili"]["lng"]))
long.append(float(df[df["name"] == "Tokyo Express"]["lng"]))
long.append(float(df[df["name"] == "Long Wong's"]["lng"]))
long.append(float(df[df["uid"] == "4e89be544901b86fc2a4ecd8"]["lng"]))

print("The location of the midpoint of the heat map is:", sum(lat)/len(lat), sum(long)/len(long))

The location of the midpoint of the heat map is: 33.49112718514639 -112.07669974657294


#### Plot the location of our restaruant on map

In [141]:
lat = 33.49112718514639
lng = -112.07669974657294
map_Phoenix = folium.Map(location=[lat, lng], zoom_start=13)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our New Restaruant",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Phoenix)

map_Phoenix

#### Phoenix_map_2
## Conclusion

Use Google Map to check the location: https://www.google.com/maps/place/33%C2%B029'28.1%22N+112%C2%B004'36.1%22W/@33.4907141,-112.0780428,17.25z/. I found it is surrounded by three hotels. So the location could have a large number of potential users. And it is safe as well since it is also next to a bank. Therefore, I think location [33.49112718514639, -112.07669974657294] is a great location for openning a new Chinese restaurant.

#### Phoenix_map_1 and Phoenix_map_2 can be found with the attachment and master branch of the repository.